# 🕷️ Web Scraping Code for Laptop Data

This notebook contains multiple web scraping implementations to collect laptop data from Flipkart.

## 📋 Contents:
1. **Cell 1**: Basic scraper with pagination (up to 10 pages)
2. **Cell 2**: Enhanced scraper with improved selectors (up to 15 pages)
3. **Cell 3**: Limited scraper with row limit (500 products max)
4. **Cell 4**: Data inspection and statistics

## ⚙️ Requirements:
- selenium
- beautifulsoup4
- webdriver-manager
- pandas

## 📝 Notes:
- The code uses Selenium 4 syntax (updated from deprecated methods)
- Headless mode is enabled for background execution
- Multiple fallback selectors for better compatibility
- Error handling for robust scraping
- Automated pagination with safety limits

## 🚀 Usage:
Run any of the first three cells to scrape data. Each implementation has different features:
- Choose Cell 1 for basic scraping
- Choose Cell 2 for more reliable scraping
- Choose Cell 3 if you want to limit the number of products

After scraping, run Cell 4 to inspect the collected data.


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time
import pandas as pd

# Setup Chrome options
options = Options()
options.add_argument('--headless')  # Run in headless mode (without GUI)
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--window-size=1920x1080')
options.add_argument('--disable-blink-features=AutomationControlled')
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)

# Setup the driver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# URL of the Flipkart page you want to scrape
url = 'https://www.flipkart.com/search?q=laptop'

# Navigate to the page
driver.get(url)
time.sleep(5)  # Wait for page to load

# Initialize an empty list to store product details
products = []

# Function to extract product details from the current page
def extract_product_details(soup):
    cards = soup.find_all('div', class_='tUxRFH')
    
    if not cards:
        print(f"No cards found. Trying alternative selectors...")
        cards = soup.find_all('div', {'data-id': True})
    
    print(f"Found {len(cards)} products on this page")

    for card in cards:
        try:
            model = card.find('div', {'class': "KzDlHZ"})
            model = model.text.strip() if model else 'N/A'
            
            star = card.find('div', {'class': "XQDdHH"})
            star = star.text.strip() if star else 'No rating'
            
            rating_elem = card.find('span', {'class': "Wphh3N"})
            if rating_elem:
                rating_text = rating_elem.text.strip()
                parts = rating_text.split('&')
                num_ratings = parts[0].strip() if len(parts) > 0 else 'No ratings'
                reviews = parts[1].strip() if len(parts) > 1 else 'No reviews'
            else:
                num_ratings = 'No ratings'
                reviews = 'No reviews'
            
            specs = card.find_all('li', {'class': 'rgWa7D'})
            processor = specs[0].text.strip() if len(specs) > 0 else 'N/A'
            ram = specs[1].text.strip() if len(specs) > 1 else 'N/A'
            os = specs[2].text.strip() if len(specs) > 2 else 'N/A'
            storage = specs[3].text.strip() if len(specs) > 3 else 'N/A'
            size = specs[4].text.strip() if len(specs) > 4 else 'N/A'
            
            price_elem = card.find('div', {'class': 'Nx9bqj'})
            price = price_elem.text.strip() if price_elem else 'Price not available'

            products.append({
                'Model': model,
                'Star Rating': star,
                'Number of Ratings': num_ratings,
                'Number of Reviews': reviews,
                'Processor': processor,
                'RAM': ram,
                'OS': os,
                'Storage': storage,
                'Size': size,
                'Price': price
            })
        except Exception as e:
            print(f"Error processing card: {e}")

# Scrape multiple pages (limit to prevent overloading)
max_pages = 10
page_count = 0

while page_count < max_pages:
    print(f"Scraping page {page_count + 1}...")
    
    # Retrieve page source
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'html.parser')

    # Extract product details from the current page
    extract_product_details(soup)

    try:
        # Find next button using Selenium 4 syntax
        next_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//a[@class='_9QVEpD' and contains(., 'Next')]"))
        )
        driver.execute_script("arguments[0].scrollIntoView();", next_button)
        time.sleep(1)
        next_button.click()
        time.sleep(5)  # Wait for the next page to load
        page_count += 1
    except Exception as e:
        print(f"No more pages or unable to find the Next button: {e}")
        break

# Close the driver
driver.quit()

# Convert the list of products into a DataFrame
df = pd.DataFrame(products)

# Save the DataFrame to a CSV file
df.to_csv('flipkart_laptops.csv', index=False)

print(f"Data has been saved to 'flipkart_laptops.csv'. Total products scraped: {len(products)}")


KeyboardInterrupt: 

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time
import pandas as pd

# Setup Chrome options with improved settings
options = Options()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--window-size=1920x1080')
options.add_argument('--disable-blink-features=AutomationControlled')
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36')

# Setup the driver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# URL of the Flipkart page you want to scrape
url = 'https://www.flipkart.com/search?q=laptop'

# Navigate to the page
driver.get(url)
time.sleep(5)

# Initialize an empty list to store product details
products = []

# Function to extract product details from the current page
def extract_product_details(soup):
    # Try multiple selectors for better compatibility
    cards = soup.find_all('div', class_='tUxRFH')
    
    if not cards:
        cards = soup.find_all('div', {'data-id': True})
    
    print(f"Found {len(cards)} products on this page")

    for card in cards:
        try:
            # Extract model name
            model_elem = card.find('div', {'class': "KzDlHZ"}) or card.find('a', {'class': 's1Q9rs'})
            model = model_elem.text.strip() if model_elem else 'N/A'
            
            # Extract star rating
            star_elem = card.find('div', {'class': "XQDdHH"})
            star = star_elem.text.strip() if star_elem else 'No rating'
            
            # Extract ratings and reviews
            rating_elem = card.find('span', {'class': "Wphh3N"}) or card.find('span', {'class': 'hG7V+4'})
            if rating_elem:
                rating_text = rating_elem.text.replace('\xa0', ' ')
                parts = rating_text.split('&')
                num_ratings = parts[0].strip() if len(parts) > 0 else 'No ratings'
                reviews = parts[1].strip() if len(parts) > 1 else 'No reviews'
            else:
                num_ratings = 'No ratings'
                reviews = 'No reviews'
            
            # Extract specifications
            specs = card.find_all('li', {'class': 'rgWa7D'}) or card.find_all('li')
            processor = specs[0].text.strip() if len(specs) > 0 else 'N/A'
            ram = specs[1].text.strip() if len(specs) > 1 else 'N/A'
            os = specs[2].text.strip() if len(specs) > 2 else 'N/A'
            storage = specs[3].text.strip() if len(specs) > 3 else 'N/A'
            size = specs[4].text.strip() if len(specs) > 4 else 'N/A'
            
            # Extract price
            price_elem = card.find('div', {'class': 'Nx9bqj'}) or card.find('div', {'class': '_30jeq3'})
            price = price_elem.text.strip() if price_elem else 'Price not available'

            products.append({
                'Model': model,
                'Star Rating': star,
                'Number of Ratings': num_ratings,
                'Number of Reviews': reviews,
                'Processor': processor,
                'RAM': ram,
                'OS': os,
                'Storage': storage,
                'Size': size,
                'Price': price
            })
        except Exception as e:
            print(f"Error processing card: {e}")
            continue

# Scrape multiple pages
max_pages = 15
page_count = 0

while page_count < max_pages:
    print(f"Scraping page {page_count + 1}...")
    
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'html.parser')
    extract_product_details(soup)

    try:
        # Updated method to find and click next button
        next_button = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//a[contains(@class, '_9QVEpD') and span[text()='Next']]"))
        )
        driver.execute_script("arguments[0].scrollIntoView(true);", next_button)
        time.sleep(1)
        driver.execute_script("arguments[0].click();", next_button)
        time.sleep(5)
        page_count += 1
    except Exception as e:
        print(f"Pagination ended or error: {e}")
        break

# Close the driver
driver.quit()

# Convert to DataFrame and save
df = pd.DataFrame(products)
df.to_csv('flipkart_laptops_extended.csv', index=False)

print(f"Scraping complete! Total products: {len(products)}")
print(f"Data saved to 'flipkart_laptops_extended.csv'")


No more pages or unable to find the Next button.
Data has been saved to 'flipkart_gaming_laptops.csv'.


In [ ]:
# Alternative scraper with row limit and error handling

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time
import pandas as pd

# Setup Chrome options
options = Options()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--window-size=1920x1080')
options.add_argument('--disable-blink-features=AutomationControlled')
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)

# Setup the driver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# URL to scrape
url = 'https://www.flipkart.com/search?q=laptop'

# Navigate to the page
driver.get(url)
time.sleep(5)

# Initialize list and set row limit
products = []
max_rows = 500  # Limit for the number of products to scrape

# Function to extract product details from the current page
def extract_product_details(soup):
    cards = soup.find_all('div', class_='tUxRFH')
    
    if not cards:
        cards = soup.find_all('div', {'data-id': True})
    
    print(f"Found {len(cards)} products on this page")

    for card in cards:
        # Check if we've reached the limit
        if len(products) >= max_rows:
            return False
            
        try:
            model_elem = card.find('div', {'class': "KzDlHZ"})
            model = model_elem.text.strip() if model_elem else 'N/A'
            
            star_elem = card.find('div', {'class': "XQDdHH"})
            star = star_elem.text.strip() if star_elem else 'No rating'
            
            rating_elem = card.find('span', {'class': "Wphh3N"})
            if rating_elem:
                rating_text = rating_elem.text.replace('\xa0', ' ')
                parts = rating_text.split('&')
                num_ratings = parts[0].strip() if len(parts) > 0 else 'No ratings'
                reviews = parts[1].strip() if len(parts) > 1 else 'No reviews'
            else:
                num_ratings = 'No ratings'
                reviews = 'No reviews'
            
            specs = card.find_all('li', {'class': 'rgWa7D'})
            processor = specs[0].text.strip() if len(specs) > 0 else 'N/A'
            ram = specs[1].text.strip() if len(specs) > 1 else 'N/A'
            os = specs[2].text.strip() if len(specs) > 2 else 'N/A'
            storage = specs[3].text.strip() if len(specs) > 3 else 'N/A'
            size = specs[4].text.strip() if len(specs) > 4 else 'N/A'
            
            price_elem = card.find('div', {'class': 'Nx9bqj'})
            price = price_elem.text.strip() if price_elem else 'Price not available'

            products.append({
                'Model': model,
                'Star Rating': star,
                'Number of Ratings': num_ratings,
                'Number of Reviews': reviews,
                'Processor': processor,
                'RAM': ram,
                'OS': os,
                'Storage': storage,
                'Size': size,
                'Price': price
            })
        except Exception as e:
            print(f"Error processing card: {e}")
            continue
    
    return True

# Scrape multiple pages
page_count = 0

while len(products) < max_rows:
    print(f"Scraping page {page_count + 1}... (Products collected: {len(products)}/{max_rows})")
    
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'html.parser')
    
    # Extract products and check if we should continue
    should_continue = extract_product_details(soup)
    
    if not should_continue or len(products) >= max_rows:
        print(f"Reached the maximum row limit of {max_rows}.")
        break

    try:
        # Find and click next button
        next_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//a[contains(@class, '_9QVEpD')]//span[contains(text(), 'Next')]"))
        )
        driver.execute_script("arguments[0].scrollIntoView(true);", next_button)
        time.sleep(1)
        driver.execute_script("arguments[0].click();", next_button)
        time.sleep(5)
        page_count += 1
    except Exception as e:
        print(f"Pagination ended or error: {e}")
        break

# Close the driver
driver.quit()

# Convert to DataFrame
df = pd.DataFrame(products)

# Remove duplicates if any
df = df.drop_duplicates()

# Save to CSV
df.to_csv('flipkart_laptops_limited.csv', index=False)

print(f"\nScraping complete!")
print(f"Total products collected: {len(df)}")
print(f"Data saved to 'flipkart_laptops_limited.csv'")
print(f"\nFirst few rows:")
print(df.head())


In [ ]:
# Data inspection and basic statistics

import pandas as pd

# Load the scraped data
try:
    df = pd.read_csv('flipkart_laptops.csv')
    
    print("=" * 80)
    print("DATA OVERVIEW")
    print("=" * 80)
    print(f"\nTotal records: {len(df)}")
    print(f"\nColumn names: {list(df.columns)}")
    print(f"\nData types:\n{df.dtypes}")
    print(f"\nMissing values:\n{df.isnull().sum()}")
    
    print("\n" + "=" * 80)
    print("FIRST 10 ROWS")
    print("=" * 80)
    print(df.head(10))
    
    print("\n" + "=" * 80)
    print("BASIC STATISTICS")
    print("=" * 80)
    print(df.describe())
    
    # Save a cleaned version
    df_clean = df.dropna()
    df_clean.to_csv('uncleaned_file.csv', index=False)
    print(f"\n✓ Data saved to 'uncleaned_file.csv'")
    
except FileNotFoundError:
    print("Error: CSV file not found. Please run the scraping cells first!")
except Exception as e:
    print(f"Error: {e}")


KeyboardInterrupt: 